# Code generator

Neural network that generates Golang code

In [39]:
import json
import numpy as np
import os
import tensorflow as tf

In [37]:
class Dataset(object):
    dataset_file = 'dataset.dt'
    dataset_folder = 'dataset'
    
    def __init__(self):
        dataset = self.load_dataset()
        self.encoded, self.vocab_to_int, self.int_to_vocab = dataset
        
    def load_dataset(self):
        if self.exists_clean_dataset():
            return self.load_clean_dataset()
        dataset = self.create_dataset()
        self.save_clean_dataset(dataset)
        return dataset

    def exists_clean_dataset(self):
        return os.path.isfile(self.dataset_file)

    def load_clean_dataset(self):
        with open(self.dataset_file, "r") as d:
            dataset = json.loads(d.read())
            return (
                np.array(dataset["encoded"]),
                dataset["vocab_to_int"],
                dataset["int_to_vocab"]
            )
        
    def create_dataset(self):
        file_names = self.get_file_names()
        content = self.get_content_from_files(file_names)
        return self.create_vocab(content)
            
    def get_file_names(self):
        files = os.listdir(self.dataset_folder)
        return [f for f in files if f.endswith('.go')]
    
    def get_content_from_files(self, file_names):
        content = ''
        for name in file_names:
            with open(os.path.join(self.dataset_folder, name), 'r') as f:
                content += f.read() + '\n'
        return content
    
    def create_vocab(self, content):
        vocab = set(content)
        vocab_to_int = {c: i for i, c in enumerate(vocab)}
        int_to_vocab = dict(enumerate(vocab))
        encoded = np.array([vocab_to_int[c] for c in content], dtype=np.int32)
        return encoded, vocab_to_int, int_to_vocab
    
    def save_clean_dataset(self, dataset):
        encoded, v_to_int, int_to_v = dataset
        with open(self.dataset_file, "w+") as d:
            d.write(json.dumps({
                'encoded' : [int(i) for i in encoded], 
                'vocab_to_int' : v_to_int,
                'int_to_vocab' : int_to_v
            }))

In [38]:
class Model(object):
    batch_size = None
    sequence_size = None
    
    def __init__(self):
        self.build_graph()
        
    def build_graph(self):
        self.build_inputs()
        self.build_lstm()
        self.build_output()
        self.build_loss()
        self.build_optimizer()
        
    def build_inputs(self):
        self.inputs = tf.placeholder(tf.int32, [None, self.sequence_size], name='inputs')
        self.targ = tf.placeholder(tf.int32, [None, self.sequence_size], name='inputs')
        self.inputs = tf.placeholder(tf.int32, [None, self.sequence_size], name='inputs')
        
Model()

AttributeError: 'Model' object has no attribute 'build_inputs'

In [4]:
class Trainer(Model):
    pass

In [5]:
class Predcitor(Model):
    pass